## 1. Understand the dataset

## 2. Combine all dataset

In [23]:
import os

import pandas as pd

excel_data: list[dict[str, pd.DataFrame]] = []

for root, dirs, files in os.walk('./certified_test'):
    for file in files:
        if file.endswith('.xlsx'):
            path = os.path.join(root, file)

            excel_data.append(pd.read_excel(path, sheet_name=None))

In [24]:
data_frames = []

for item in excel_data:
    for sheet, data_frame in item.items():
        data_frame.drop(data_frame[-1:].index, inplace=True)
        data_frames.append(data_frame)

In [25]:
data = pd.concat(data_frames)

## 3. Check and change data type of each column to appropriate type

## 4. Replace ( – ) by 0

Both steps done here

In [26]:
for c in data.columns:
    if c == 'Grade/100.00' or c.startswith('Q.'):
        data[c] = data[c].map(lambda x: float(x) if x != '-' else 0)

## 5. Drop duplicates

In [27]:
data = data.drop_duplicates()

## 6. Create new column ‘Label’ – Passed if greater than or equal to 60, Failed if less than 60

In [36]:
data['Label'] = data['Grade/100.00'].map(lambda x: 'Passed' if x >= 60 else 'Failed')

data[:3].loc[:,['Grade/100.00', 'Label']] # check

,Grade/100.00,Label
0,58.13,Failed
1,69.38,Passed
2,54.31,Failed


## 7. Rename Grade100/00 to Total Score

In [37]:
data = data.rename(columns={'Grade/100.00': 'Total Score'})

data[:1]

,Surname,First name,Email address,State,Started on,Completed,Time taken,Total Score,Q. 1 /1.00,Q. 2 /1.00,...,Q. 92 /1.00,Q. 93 /1.00,Q. 94 /1.00,Q. 95 /1.00,Q. 96 /1.00,Q. 97 /1.00,Q. 98 /1.00,Q. 99 /1.00,Q. 100 /1.00,Label
0,វង្សច័ន្ទភូមាន,គៀត,D220131@DSEERA22.com,Finished,23 December 2022 8:14 AM,23 December 2022 9:52 AM,1 hour 37 mins,58.13,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,Failed


## 8. Detect outliers in Total Score column – Keep outliers in our case

In [ ]:
# Find the average at the lower and upper 25%
q1 = data['Total Score'].quantile(0.25)
q3 = data['Total Score'].quantile(0.75)

iqr = q3 - q1

# Get the bounds
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

outliers = data[(data['Total Score'] < lower_bound) | (data['Total Score'] > upper_bound)]

outliers

,Surname,First name,Email address,State,Started on,Completed,Time taken,Total Score,Q. 1 /1.00,Q. 2 /1.00,...,Q. 92 /1.00,Q. 93 /1.00,Q. 94 /1.00,Q. 95 /1.00,Q. 96 /1.00,Q. 97 /1.00,Q. 98 /1.00,Q. 99 /1.00,Q. 100 /1.00,Label
622,សុខនី,ជា,dp23422@DSEPROVINCE23.com,In progress,3 August 2023 8:01 AM,-,-,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Failed
647,រតនា,សឿន,dp23470@DSEPROVINCE23.com,In progress,3 August 2023 10:18 AM,-,-,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Failed
299,ចាន់ម៉ុល,ឌិន,dp231422@DSEPROVINCE23.com,Finished,2 March 2023 8:34 AM,2 March 2023 10:36 AM,2 hours 2 mins,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Failed


## 9. Drop row where State is not Finished

In [38]:
data = data.drop(data.loc[data['State'] != "Finished"].index)
data = data.reindex()

## 10. Deal with missing values if any

In [ ]:
print(len(data))
data.dropna()
print(len(data))

2066
2066


No rows dropped means there were no missing values

## Save to a file

In [ ]:
data.to_excel('certified_test_output.xlsx')